## gen_trainval_list.py

In [1]:
import os
import glob
import random

In [21]:
root = 'maize_counting_dataset'
image_folder = 'images'
label_folder = 'labels'
train = 'train'
val = 'val'

In [22]:
train_path = os.path.join(root, train)
with open('train.txt', 'w') as f:
    for image_path in glob.glob(os.path.join(train_path, image_folder, '*.JPG')):
        im_path = image_path.replace(root, '')
        gt_path = im_path.replace(image_folder, label_folder).replace('.JPG', '.xml')
        f.write(im_path+'\t'+gt_path+'\n')

In [23]:
val_path = os.path.join(root, val)
with open('val.txt', 'w') as f:
    for image_path in glob.glob(os.path.join(val_path, image_folder, '*.JPG')):
        im_path = image_path.replace(root, '')
        gt_path = im_path.replace(image_folder, label_folder).replace('.JPG', '.xml')
        f.write(im_path+'\t'+gt_path+'\n')

## hldataset.py

In [10]:
import os
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd 
import random
import numpy as np
from PIL import Image
import cv2
import h5py
import scipy.io as sio
from scipy.ndimage.filters import gaussian_filter
from skimage import util
from skimage.measure import label
from skimage.measure import regionprops
import xml.etree.ElementTree as ET

import torch
from torch.utils.data import Dataset
from torchvision import transforms
import torch.nn.functional as F

In [11]:
def read_image(x):
    img_arr = np.array(Image.open(x))
    if len(img_arr.shape) == 2:  # grayscale
        img_arr = np.tile(img_arr, [3, 1, 1]).transpose(1, 2, 0)
    return img_arr

In [12]:
class RandomCrop(object):
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):

        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        h, w = image.shape[:2]

        if isinstance(self.output_size, tuple):
            new_h = min(self.output_size[0], h)
            new_w = min(self.output_size[1], w)
            assert (new_h, new_w) == self.output_size
        else:
            crop_size = min(self.output_size, h, w)
            assert crop_size == self.output_size
            new_h = new_w = crop_size
        if gtcount > 0:
            mask = target > 0
            ch, cw = int(np.ceil(new_h / 2)), int(np.ceil(new_w / 2))
            mask_center = np.zeros((h, w), dtype=np.uint8)
            mask_center[ch:h-ch+1, cw:w-cw+1] = 1
            mask = (mask & mask_center)
            idh, idw = np.where(mask == 1)
            if len(idh) != 0:
                ids = random.choice(range(len(idh)))
                hc, wc = idh[ids], idw[ids]
                top, left = hc-ch, wc-cw
            else:
                top = np.random.randint(0, h-new_h+1)
                left = np.random.randint(0, w-new_w+1)
        else:
            top = np.random.randint(0, h-new_h+1)
            left = np.random.randint(0, w-new_w+1)

        image = image[top:top+new_h, left:left+new_w, :]
        target = target[top:top+new_h, left:left+new_w]

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [13]:
class RandomFlip(object):
    def __init__(self):
        pass

    def __call__(self, sample):
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        do_mirror = np.random.randint(2)
        if do_mirror:
            image = cv2.flip(image, 1)
            target = cv2.flip(target, 1)
        return {'image': image, 'target': target, 'gtcount': gtcount}

In [14]:
class Normalize(object):

    def __init__(self, scale, mean, std):
        self.scale = scale
        self.mean = mean
        self.std = std

    def __call__(self, sample):
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        image, target = image.astype('float32'), target.astype('float32')

        # pixel normalization
        image = (self.scale * image - self.mean) / self.std

        image, target = image.astype('float32'), target.astype('float32')

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [15]:
class ZeroPadding(object):
    def __init__(self, psize=32):
        self.psize = psize

    def __call__(self, sample):
        psize =  self.psize

        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        h,w = image.size()[-2:]
        ph,pw = (psize-h%psize),(psize-w%psize)
        # print(ph,pw)

        (pl, pr) = (pw//2, pw-pw//2) if pw != psize else (0, 0)
        (pt, pb) = (ph//2, ph-ph//2) if ph != psize else (0, 0)
        if (ph!=psize) or (pw!=psize):
            tmp_pad = [pl, pr, pt, pb]
            # print(tmp_pad)
            image = F.pad(image,tmp_pad)
            target = F.pad(target,tmp_pad)

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [16]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __init__(self):
        pass

    def __call__(self, sample):
        # swap color axis
        # numpy image: H x W x C
        # torch image: C X H X W
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        image = image.transpose((2, 0, 1))
        target = np.expand_dims(target, axis=2)
        target = target.transpose((2, 0, 1))
        image, target = torch.from_numpy(image), torch.from_numpy(target)
        return {'image': image, 'target': target, 'gtcount': gtcount}

In [17]:
class MaizeTasselDataset(Dataset):
    def __init__(self, data_dir, data_list, ratio, train=True, transform=None):
        self.data_dir = data_dir
        self.data_list = [name.split('\t') for name in open(data_list).read().splitlines()]
        self.ratio = ratio
        self.train = train
        self.transform = transform
        self.image_list = []
        
        # store images and generate ground truths
        self.images = {}
        self.targets = {}
        self.gtcounts = {}
        self.dotimages = {}

    def bbs2points(self, bbs):    
        points = []
        for bb in bbs:
            x1, y1, w, h = [float(b) for b in bb]
            x2, y2 = x1+w-1, y1+h-1
            x, y = np.round((x1+x2)/2).astype(np.int32), np.round((y1+y2)/2).astype(np.int32)
            points.append([x, y])
        return points

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        file_name = self.data_list[idx]
        self.image_list.append(file_name[0])
        if file_name[0] not in self.images:
            image = read_image(self.data_dir+file_name[0])
            annotation = sio.loadmat(self.data_dir+file_name[1])
            h, w = image.shape[:2]
            nh = int(np.ceil(h * self.ratio))
            nw = int(np.ceil(w * self.ratio))
            image = cv2.resize(image, (nw, nh), interpolation = cv2.INTER_CUBIC)
            target = np.zeros((nh, nw), dtype=np.float32)
            dotimage = image.copy()
            if annotation['annotation'][0][0][1] is not None:
                bbs = annotation['annotation'][0][0][1]
                gtcount = bbs.shape[0]
                pts = self.bbs2points(bbs)
                for pt in pts:
                    pt[0], pt[1] = int(pt[0] * self.ratio), int(pt[1] * self.ratio)
                    target[pt[1], pt[0]] = 1
                    cv2.circle(dotimage, (pt[0], pt[1]), int(24 * self.ratio) , (255, 0, 0), -1)
            else:
                gtcount = 0
            target = gaussian_filter(target, 80 * self.ratio)

            # plt.imshow(target, cmap=cm.jet)
            # plt.show()
            # print(target.sum())

            self.images.update({file_name[0]:image})
            self.targets.update({file_name[0]:target})
            self.gtcounts.update({file_name[0]:gtcount})
            self.dotimages.update({file_name[0]:dotimage})

        
        sample = {
            'image': self.images[file_name[0]], 
            'target': self.targets[file_name[0]], 
            'gtcount': self.gtcounts[file_name[0]]
        }

        if self.transform:
            sample = self.transform(sample)

        return sample

In [24]:
dataset = MaizeTasselDataset(
        data_dir='maize_counting_dataset', 
        data_list='maize_counting_dataset/train.txt',
        ratio=0.167, 
        train=True, 
        transform=transforms.Compose([
            ToTensor()]
        )
    )

In [25]:
dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        num_workers=0
    )

In [27]:
print(len(dataloader))
mean = 0.
std = 0.
for i, data in enumerate(dataloader, 0):
        images, targets = data['image'], data['target']
        bs = images.size(0)
        images = images.view(bs, images.size(1), -1).float()
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        print(images.size())
        print(i) 
mean /= len(dataloader)
std /= len(dataloader)
print(mean/255.)
print(std/255.)

186
torch.Size([1, 3, 278770])
0
torch.Size([1, 3, 278770])
1
torch.Size([1, 3, 278770])
2
torch.Size([1, 3, 278770])
3
torch.Size([1, 3, 278770])
4
torch.Size([1, 3, 222530])
5
torch.Size([1, 3, 278770])
6
torch.Size([1, 3, 222530])
7
torch.Size([1, 3, 278770])
8
torch.Size([1, 3, 278770])
9
torch.Size([1, 3, 278770])
10
torch.Size([1, 3, 278770])
11
torch.Size([1, 3, 278770])
12
torch.Size([1, 3, 278770])
13
torch.Size([1, 3, 278770])
14
torch.Size([1, 3, 278770])
15
torch.Size([1, 3, 278770])
16
torch.Size([1, 3, 278770])
17
torch.Size([1, 3, 278770])
18
torch.Size([1, 3, 278770])
19
torch.Size([1, 3, 278770])
20
torch.Size([1, 3, 278770])
21
torch.Size([1, 3, 278770])
22
torch.Size([1, 3, 278770])
23
torch.Size([1, 3, 278770])
24
torch.Size([1, 3, 278770])
25
torch.Size([1, 3, 278770])
26
torch.Size([1, 3, 278770])
27
torch.Size([1, 3, 278770])
28
torch.Size([1, 3, 278770])
29
torch.Size([1, 3, 278770])
30
torch.Size([1, 3, 278770])
31
torch.Size([1, 3, 278770])
32
torch.Size([1, 3

## utils.py

In [30]:
import scipy
import numpy as np
import math
import cv2 as cv
from scipy.ndimage import gaussian_filter, morphology
from skimage.measure import label, regionprops
from sklearn import linear_model
import matplotlib.pyplot as plt

In [31]:
def compute_mae(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    mae = np.mean(np.abs(diff))
    return mae


def compute_mse(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    mse = np.sqrt(np.mean((diff ** 2)))
    return mse

def compute_relerr(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    diff = diff[gt > 0]
    gt = gt[gt > 0]
    if (diff is not None) and (gt is not None):
        rmae = np.mean(np.abs(diff) / gt) * 100
        rmse = np.sqrt(np.mean(diff**2 / gt**2)) * 100
    else:
        rmae = 0
        rmse = 0
    return rmae, rmse


def rsquared(pd, gt):
    """ Return R^2 where x and y are array-like."""
    pd, gt = np.array(pd), np.array(gt)
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(pd, gt)
    return r_value**2


def dense_sample2d(x, sx, stride):
    (h,w) = x.shape[:2]
    #idx_img = np.array([i for i in range(h*w)]).reshape(h,w)
    idx_img = np.zeros((h,w),dtype=float)
    
    th = [i for i in range(0, h-sx+1, stride)]
    tw = [j for j in range(0, w-sx+1, stride)]
    norm_vec = np.zeros(len(th)*len(tw))

    for i in th:
        for j in tw:
            idx_img[i:i+sx,j:j+sx] = idx_img[i:i+sx,j:j+sx]+1

    # # plot redundancy map
    # import os
    # import matplotlib.pyplot as plt
    # cmap = plt.cm.get_cmap('hot')
    # idx_img = idx_img / (idx_img.max())
    # idx_img = cmap(idx_img) * 255.
    # plt.figure()
    # plt.imshow(idx_img.astype(np.uint8))
    # plt.axis('off')
    # plt.savefig(os.path.join('redundancy_map.pdf'), bbox_inches='tight', dpi = 300)
    # plt.close()
   
    idx_img = 1/idx_img
    idx_img = idx_img/sx/sx
    #line order
    idx = 0
    for i in th:
        for j in tw:
            norm_vec[idx] =idx_img[i:i+sx,j:j+sx].sum()
            idx+=1
    
    return norm_vec


def recover_countmap(pred, image, patch_sz, stride):
    pred = pred.reshape(-1)
    imH, imW = image.shape[2:4]
    cntMap = np.zeros((imH, imW), dtype=float)
    norMap = np.zeros((imH, imW), dtype=float)
    
    H = np.arange(0, imH - patch_sz + 1, stride)
    W = np.arange(0, imW - patch_sz + 1, stride)
    cnt = 0
    for h in H:
        for w in W:
            pixel_cnt = pred[cnt] / patch_sz / patch_sz
            cntMap[h:h+patch_sz, w:w+patch_sz] += pixel_cnt
            norMap[h:h+patch_sz, w:w+patch_sz] += np.ones((patch_sz,patch_sz))
            cnt += 1
    return cntMap / (norMap + 1e-12)

## hlnet.py

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [33]:
class Encoder(nn.Module):
    def __init__(self, arc='tasselnetv2'):
        super(Encoder, self).__init__()
        if arc == 'tasselnetv2':
            self.encoder = nn.Sequential(
                nn.Conv2d(3, 16, 3, padding=1, bias=False),
                nn.BatchNorm2d(16),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(16, 32, 3, padding=1, bias=False),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(32, 64, 3, padding=1, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(64, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
            )
        elif arc == 'tasselnetv2plus':
            self.encoder = nn.Sequential(
                nn.Conv2d(3, 16, 3, padding=1, bias=False),
                nn.BatchNorm2d(16),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(16, 32, 3, padding=1, bias=False),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(32, 64, 3, padding=1, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(64, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True)
            )
        else:
            raise NotImplementedError

    def forward(self, x):
        x = self.encoder(x)
        return x

In [34]:
class Counter(nn.Module):
    def __init__(self, arc='tasselnetv2', input_size=64, output_stride=8):
        super(Counter, self).__init__()
        k = int(input_size / 8)
        avg_pool_stride = int(output_stride / 8)

        if arc == 'tasselnetv2':
            self.counter = nn.Sequential(
                nn.Conv2d(128, 128, (k, k), bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, 1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 1, 1)
            )
        elif arc == 'tasselnetv2plus':
            self.counter = nn.Sequential(
                nn.AvgPool2d((k, k), stride=avg_pool_stride),
                nn.Conv2d(128, 128, 1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 1, 1)
            )
        else:
            raise NotImplementedError

    def forward(self, x):
        x = self.counter(x)
        return x

In [35]:
class Normalizer:
    @staticmethod
    def cpu_normalizer(x, imh, imw, insz, os):
        # CPU normalization
        bs = x.size()[0]
        normx = np.zeros((imh, imw))
        norm_vec = dense_sample2d(normx, insz, os).astype(np.float32)
        x = x.cpu().detach().numpy().reshape(bs, -1) * norm_vec
        return x
    
    @staticmethod
    def gpu_normalizer(x, imh, imw, insz, os):
        _, _, h, w = x.size()            
        accm = torch.cuda.FloatTensor(1, insz*insz, h*w).fill_(1)           
        accm = F.fold(accm, (imh, imw), kernel_size=insz, stride=os)
        accm = 1 / accm
        accm /= insz**2
        accm = F.unfold(accm, kernel_size=insz, stride=os).sum(1).view(1, 1, h, w)
        x *= accm
        return x.squeeze().cpu().detach().numpy()

In [36]:
class CountingModels(nn.Module):
    def __init__(self, arc='tasselnetv2', input_size=64, output_stride=8):
        super(CountingModels, self).__init__()
        self.input_size = input_size
        self.output_stride = output_stride

        self.encoder = Encoder(arc)
        self.counter = Counter(arc, input_size, output_stride)
        if arc == 'tasselnetv2':
            self.normalizer = Normalizer.cpu_normalizer
        elif arc == 'tasselnetv2plus':
            self.normalizer = Normalizer.gpu_normalizer
        
        self.weight_init()

    def forward(self, x, is_normalize=True):
        imh, imw = x.size()[2:]
        x = self.encoder(x)
        x = self.counter(x)
        if is_normalize:
            x = self.normalizer(x, imh, imw, self.input_size, self.output_stride)
        return x

    def weight_init(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                # nn.init.kaiming_uniform_(
                #         m.weight, 
                #         mode='fan_in', 
                #         nonlinearity='relu'
                #         )
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

In [45]:
from time import time

insz, os = 64, 8
imH, imW = 1080, 1920
net = CountingModels(arc='tasselnetv2', input_size=insz, output_stride=os).cuda()
with torch.no_grad():
    net.eval()
    x = torch.randn(1, 3, imH, imW).cuda()
    y = net(x)
    print(y.shape)    

with torch.no_grad():
    frame_rate = np.zeros((100, 1))
    for i in range(100):
        x = torch.randn(1, 3, imH, imW).cuda()
        torch.cuda.synchronize()
        start = time()

        y = net(x)

        torch.cuda.synchronize()
        end = time()

        running_frame_rate = 1 * float(1 / (end - start))
        frame_rate[i] = running_frame_rate
    print(np.mean(frame_rate))

AssertionError: Torch not compiled with CUDA enabled

## hltrainval.py

In [42]:
import os
import argparse
from time import time

import cv2 as cv
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

plt.switch_backend('agg')
# from skimage.measure import compare_psnr updated to peak_signal_noise_ratio
# from skimage.measure import compare_ssim updated to structural_similarity
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity
import torch.backends.cudnn as cudnn

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR, MultiStepLR

In [44]:
# prevent dataloader deadlock, uncomment if deadlock occurs
# cv.setNumThreads(0)
cudnn.enabled = True

# constant
IMG_SCALE = 1. / 255
IMG_MEAN = [.3405, .4747, .2418]
IMG_STD = [1, 1, 1]
SCALES = [0.7, 1, 1.3]
SHORTER_SIDE = 224

# system-related parameters
DATA_DIR = 'maize_counting_dataset'
DATASET = 'mtc'
EXP = 'tasselnetv2_rf110_i64o8_r0125_crop256_lr-2_bs9_epoch500'
DATA_LIST = 'maize_counting_dataset/train.txt'
DATA_VAL_LIST = 'maize_counting_dataset/test.txt'

RESTORE_FROM = 'model_best.pth.tar'
SNAPSHOT_DIR = './snapshots'
RESULT_DIR = './results'

# model-related parameters
INPUT_SIZE = 64
OUTPUT_STRIDE = 8
MODEL = 'tasselnetv2'
RESIZE_RATIO = 0.125

# training-related parameters
OPTIMIZER = 'sgd'  # choice in ['sgd', 'adam']
BATCH_SIZE = 9
CROP_SIZE = (256, 256)
LEARNING_RATE = 1e-2
MILESTONES = [200, 400]
MOMENTUM = 0.95
MULT = 1
NUM_EPOCHS = 500
NUM_CPU_WORKERS = 0
PRINT_EVERY = 1
RANDOM_SEED = 6
WEIGHT_DECAY = 5e-4
VAL_EVERY = 1

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# add a new entry here if creating a new data loader
dataset_list = {
    'mtc': MaizeTasselDataset
}